# Step 2: Generate data samples

In this section, we will use the amplitude model that we created with the
expert system to generate a data sample via hit & miss Monte Carlo. This
requires us to work with the User Interface to ComPWA, the C++ back-end of
`pycompwa`.

## 2.1 Generate

In the previous step, we created an amplitude module and exported it as an
XML file. We now need to go through a few steps to generate a phase space
sample and a data sample for the decay $J/\psi \rightarrow \gamma\pi^0\pi^0$:

Now all building blocks for generating our data sample are at hand and you
can define a data sample of arbitrary size!

In [ ]:
import pycompwa.ui as pwa  # interface to ComPWA

In [ ]:
# specify kinematics
particle_list = pwa.read_particles('model.xml')
kinematics = pwa.create_helicity_kinematics('model.xml', particle_list)

# specify generators
generator = pwa.EvtGenGenerator(
    kinematics.get_particle_state_transition_kinematics_info())
random_generator = pwa.StdUniformRealGenerator(12345)

# generate phase space sample
phsp_sample = pwa.generate_phsp(int(1e5), generator, random_generator)

# create intensity profile
intensity_builder = pwa.IntensityBuilderXML(
    'model.xml', particle_list, kinematics, phsp_sample)
intensity = intensity_builder.create_intensity()

# generate data
data_sample = pwa.generate(
    int(1e4), kinematics, generator, intensity, random_generator)

## 2.2 Exporting and importing

In [ ]:
# to ROOT
pwa.write_root_data(data_sample, output_file='generated_data.root')
pwa.write_root_data(phsp_sample, output_file='generated_phsp.root')
# to ASCII
pwa.write_ascii_data(data_sample, output_file='generated_data.dat')
pwa.write_ascii_data(phsp_sample, output_file='generated_phsp.dat')

The syntax for importing parallels that:

In [ ]:
imported_data = pwa.read_root_data(input_file='generated_data.root')
imported_phsp = pwa.read_ascii_data(input_file='generated_phsp.dat')

print('Imported data events:', len(imported_data.events))
print('Imported phsp events:', len(imported_phsp.events))

first_event = imported_data.events[0]
print('First data event:')
print(first_event.four_momenta)
print(first_event.weight)

Note that the ASCII (``.dat``) files contain a header section that informs
you about the final state of the file. You will have to prepend this section
to the file yourself if you want to use an ASCII file that was exported from
another framework.

In [ ]:
with open('generated_data.dat') as f:
    for _ in range(5):
        print(f.readline()[:-1])